<a href="https://colab.research.google.com/github/rbfl1227/Numerical-Matheamatics-and-Computing/blob/main/ch2_%EC%84%A0%ED%98%95_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **2.1 순수 가우스 소거법**

In [2]:
import numpy as np

In [16]:
def naive_gauss(A, b):
    """
    순수 가우스 소거법 (Naive Gauss)
    A: 계수 행렬 (n x n)
    b: 상수 벡터 (n x 1)
    """
    n = len(A)
    A = A.astype(float)
    b = b.astype(float)

    for k in range(n-1):
        for i in range(k+1, n):
            xmult = A[i, k] / A[k, k]
            A[i, k] = xmult
            for j in range(k+1, n):
                A[i, j] -= xmult * A[k, j]
            b[i] -= xmult * b[k]

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum_ = b[i]
        for j in range(i+1, n):
            sum_ -= A[i, j] * x[j]
        x[i] = sum_ / A[i, i]

    return x


In [17]:
#1
def naive_gauss(n, A, b):
    """
    순수 가우스 소거법(Naive Gauss)
    A: 계수 행렬 (n x n)
    b: 상수 벡터 (n x 1)
    """
    A = A.astype(float)
    b = b.astype(float)

    for k in range(n-1):
        for i in range(k+1, n):
            xmult = A[i, k] / A[k, k]
            A[i, k] = xmult
            for j in range(k+1, n):
                A[i, j] -= xmult * A[k, j]
            b[i] -= xmult * b[k]

    x = np.zeros(n)
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum_ = b[i]
        for j in range(i+1, n):
            sum_ -= A[i, j] * x[j]
        x[i] = sum_ / A[i, i]

    return x

    def test_nge():
      for n in range(4, 11):  # n = 4,5,6,7,8,9,10 테스트
          A = np.zeros((n, n))
          b = np.zeros(n)

          for i in range(n):
            for j in range(n):
                A[i, j] = 1 / (i + j + 1)
            b[i] = ((1 + i) ** n - 1) / i if i != 0 else 0

          print("\n======================================")
          print(f"테스트 중: n = {n}")
          x = naive_gauss(n, A, b)
          print(f"n = {n}, 해: {x}")
          print("======================================\n")

    test_nge()


In [22]:
#3
n = 4
A = np.zeros((n, n))
b = np.zeros(n)
for i in range(n):
    for j in range(n):
        A[i, j] = i + j
    b[i] = i + 1

after_A, indices = gauss(n, A)
x = solve(n, after_A, indices, b)
print("3번 문제: n=4에 대한 해 x:")
print(x)

3번 문제: n=4에 대한 해 x:
[       -inf 15.94444444 -4.05555556 -2.27777778]


<ipython-input-20-3e08216216d2>:20: RuntimeWarning: divide by zero encountered in scalar divide
  x[i] = sum_ / A[l[i], i]


In [23]:
#5
n = 3
A = np.array([[2, -1, 1],
              [3, 3, 9],
              [3, 3, 5]], dtype=float)
b = np.array([2, -1, 4], dtype=float)

after_A, indices = gauss(n, A)
if after_A is not None:
    x = solve(n, after_A, indices, b)
    print("5번 문제: 해 x:")
    print(x)

5번 문제: 해 x:
[ 7.57142857 10.71428571 -2.42857143]


### **2.2 비례 조정 부분 피벗팅**

In [18]:
import numpy as np

In [19]:
def gauss(n, A):
    """
    비례 조정 부분 피벗팅을 사용한 가우스 소거법
    A: 계수 행렬 (n x n)
    """
    A = A.astype(float)
    l = np.arange(n)  # 인덱스 배열
    s = np.zeros(n)  # 비례 조정을 위한 배열

    # 비례 조정 스케일링 벡터 초기화
    for i in range(n):
        smax = 0
        for j in range(n):
            smax = max(smax, abs(A[i, j]))
        s[i] = smax

    # 전진 소거
    for k in range(n-1):
        rmax = 0
        for i in range(k, n):
            r = abs(A[i, k] / s[i])
            if r > rmax:
                rmax = r
                max_index = i

        # 행 교환
        l[k], l[max_index] = l[max_index], l[k]
        A[[k, max_index]] = A[[max_index, k]]

        for i in range(k+1, n):
            xmult = A[i, k] / A[k, k]
            A[i, k] = xmult
            for j in range(k+1, n):
                A[i, j] -= xmult * A[k, j]

    return A, l


In [20]:
def solve(n, A, l, b):
    """
    역대입을 수행하여 해를 구하는 함수
    A: 가우스 소거법을 적용한 행렬
    l: 행 교환 인덱스 배열
    b: 상수 벡터
    """
    # 순서를 반영한 b 벡터 조정
    for k in range(n-1):
        for i in range(k+1, n):
            b[l[i]] -= A[l[i], k] * b[l[k]]

    # 역대입 과정
    x = np.zeros(n)
    x[n-1] = b[l[n-1]] / A[l[n-1], n-1]
    for i in range(n-2, -1, -1):
        sum_ = b[l[i]]
        for j in range(i+1, n):
            sum_ -= A[l[i], j] * x[j]
        x[i] = sum_ / A[l[i], i]

    return x

In [21]:
A = np.array([[3, -0.1, -0.2],
              [0.1, 7, -0.3],
              [0.3, -0.2, 10]], dtype=float)
b = np.array([7.85, -19.3, 71.4], dtype=float)

after_A, indices = gauss(len(A), A)
x = solve(len(A), after_A, indices, b)

print("비례 조정 후 행렬 A:")
print(after_A)
print("인덱스 배열:")
print(indices)
print("해 x:")
print(x)


비례 조정 후 행렬 A:
[[ 3.         -0.1        -0.2       ]
 [ 0.03333333  7.00333333 -0.29333333]
 [ 0.1        -0.02712994 10.01204188]]
인덱스 배열:
[0 1 2]
해 x:
[ 3.  -2.5  7. ]


### **2.3 3중 대각 시스템과 밴드 시스템**

In [24]:
def tri(n, a, d, c, b):
    """
    3중 대각 행렬 시스템을 푸는 알고리즘 (TDMA)
    a: 아랫 대각 요소 (n-1)
    d: 주 대각 요소 (n)
    c: 윗 대각 요소 (n-1)
    b: 우변 벡터 (n)
    """
    a = np.array(a, dtype=float)
    d = np.array(d, dtype=float)
    c = np.array(c, dtype=float)
    b = np.array(b, dtype=float)

    # 전진 소거 과정
    for i in range(1, n):
        xmult = a[i-1] / d[i-1]
        d[i] -= xmult * c[i-1]
        b[i] -= xmult * b[i-1]

    # 역대입 과정
    x = np.zeros(n)
    x[-1] = b[-1] / d[-1]
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - c[i] * x[i+1]) / d[i]

    return x

In [25]:
# 3중 대각 행렬 시스템 테스트
a = [1, 1, 1, 1]
d = [4, 4, 4, 4, 4]
c = [1, 1, 1, 1]
b = [5, 5, 5, 5, 5]
n = len(d)

x = tri(n, a, d, c, b)
print("3중 대각 행렬 시스템 해 x:")
print(x)


3중 대각 행렬 시스템 해 x:
[1.05769231 0.76923077 0.86538462 0.76923077 1.05769231]


In [36]:
def penta(n, e, a, d, c, f, b):
    """
    5중 대각 행렬 시스템을 푸는 알고리즘 (Penta-Diagonal System)
    e: 하단 2번째 대각선 요소 (n-2)
    a: 하단 1번째 대각선 요소 (n-1)
    d: 주 대각선 요소 (n)
    c: 상단 1번째 대각선 요소 (n-1)
    f: 상단 2번째 대각선 요소 (n-2)
    b: 우변 벡터 (n)
    """
    e = np.array(e, dtype=float)
    a = np.array(a, dtype=float)
    d = np.array(d, dtype=float)
    c = np.array(c, dtype=float)
    f = np.array(f, dtype=float)
    b = np.array(b, dtype=float)

    r = a[0]
    s = a[1]
    t = e[0]

    for i in range(1, n-1):
        xmult = r / d[i-1]
        d[i] -= xmult * c[i-1]
        c[i] -= xmult * f[i-1]
        b[i] -= xmult * b[i-1]

        xmult = t / d[i-1]
        r = s - (xmult * c[i-1])
        d[i+1] -= xmult * f[i-1]
        b[i+1] -= xmult * b[i-1]
        s = a[i+1]
        t = e[i]

    xmult = r / d[n-2]
    d[n-1] -= xmult * c[n-2]

    x = np.zeros(n)
    x[n-1] = (b[n-1] - (xmult * b[n-2])) / d[n-1]
    x[n-2] = (b[n-2] - c[n-2] * x[n-1]) / d[n-2]

    for i in range(n-3, -1, -1):
        x[i] = (b[i] - f[i] * x[i+2] - c[i] * x[i+1]) / d[i]

    return x

In [37]:
# 5중 대각 행렬 시스템 테스트
e = [0.5, 0.5]
a = [1, 1, 1]
d = [4, 4, 4, 4]
c = [1, 1, 1]
f = [0.5, 0.5]
b = [5, 5, 5, 5]
n = len(d)

x = penta(n, e, a, d, c, f, b)
print("5중 대각 행렬 시스템 해 x:")
print(x)


IndexError: index 3 is out of bounds for axis 0 with size 3

*  배열의 길이를 초과하지 않도록 조건을 추가


In [38]:
def penta(n, e, a, d, c, f, b):
    """
    5중 대각 행렬 시스템을 푸는 알고리즘 (Penta-Diagonal System)
    e: 하단 2번째 대각선 요소 (n-2)
    a: 하단 1번째 대각선 요소 (n-1)
    d: 주 대각선 요소 (n)
    c: 상단 1번째 대각선 요소 (n-1)
    f: 상단 2번째 대각선 요소 (n-2)
    b: 우변 벡터 (n)
    """
    e = np.array(e, dtype=float)
    a = np.array(a, dtype=float)
    d = np.array(d, dtype=float)
    c = np.array(c, dtype=float)
    f = np.array(f, dtype=float)
    b = np.array(b, dtype=float)

    r = a[0] if len(a) > 0 else 0
    s = a[1] if len(a) > 1 else 0
    t = e[0] if len(e) > 0 else 0

    for i in range(1, n-2):
        xmult = r / d[i-1]
        d[i] -= xmult * c[i-1]
        c[i] -= xmult * f[i-1]
        b[i] -= xmult * b[i-1]

        xmult = t / d[i-1]
        r = s - (xmult * c[i-1]) if i < len(c) else 0
        d[i+1] -= xmult * f[i-1] if i+1 < len(d) else 0
        b[i+1] -= xmult * b[i-1] if i+1 < len(b) else 0
        s = a[i+1] if i+1 < len(a) else 0
        t = e[i] if i < len(e) else 0

    xmult = r / d[n-2]
    d[n-1] -= xmult * c[n-2]

    x = np.zeros(n)
    x[n-1] = (b[n-1] - (xmult * b[n-2])) / d[n-1]
    x[n-2] = (b[n-2] - c[n-2] * x[n-1]) / d[n-2]

    for i in range(n-3, -1, -1):
        x[i] = (b[i] - (f[i] * x[i+2] if i+2 < len(x) else 0) - c[i] * x[i+1]) / d[i]

    return x

In [39]:
# 5중 대각 행렬 시스템 테스트
e = [0.5, 0.5]
a = [1, 1, 1]
d = [4, 4, 4, 4]
c = [1, 1, 1]
f = [0.5, 0.5]
b = [5, 5, 5, 5]
n = len(d)

x = penta(n, e, a, d, c, f, b)
print("5중 대각 행렬 시스템 해 x:")
print(x)

5중 대각 행렬 시스템 해 x:
[0.97951681 0.66176471 0.84033613 1.06617647]


In [ ]:
#1번
